# Applying SQL to Real-World Problems

### Transform numeric & strings
For this exercise you are planning to run a 50% off promotion for films released prior to 2006. To prepare for this promotion you will need to return the films that qualify for this promotion, to make these titles easier to read you will convert them all to lower case. You will also need to return both the original_rate and the sale_rate.

**Original data columns from film database**
* film_id	
* title	
* description	
* release_year	
* language_id	
* rental_duration	
* rental_rate	
* length	
* replacement_cost	
* rating	
* special_features

```SQL
SELECT LOWER(title) AS title, 
  rental_rate AS original_rate, 
  rental_rate * 0.5 AS sale_rate 
FROM film
-- Filter for films prior to 2006
WHERE release_year < 2006;

--------------------------------------------------------------------------------------------
title	          original_rate	      sale_rate
adaptation holes	2.99	           1.495
annie identity      0.99	           0.495
--------------------------------------------------------------------------------------------
```
### Extract what you need
In this exercise you will practice preparing date/time elements by using the EXTRACT() function.

```SQL
SELECT payment_date,
  EXTRACT(DAY FROM payment_date) AS payment_day 
  EXTRACT(YEAR FROM payment_date) AS payment_YEAR 
FROM payment;
--------------------------------------------------------------------------------------------
payment_date	        payment_day      payment_year
2017-01-25 02:40:19+00:00	25             2017
2017-01-25 20:16:50+00:00	25             2017
2017-01-29 02:44:14+00:00	29             2017
--------------------------------------------------------------------------------------------
```
### Aggregating finances
In this exercise you would like to learn more about the differences in payments between the customers who are active and those who are not.
We have **customer** and **payment** databases as follow

* customer
    * customer_id	
    * first_name	
    * last_name	
    * email	
    * address_id	
    * active
    
* payment
     * payment_id	
     * customer_id	
     * rental_id	
     * amount	
     * payment_date

```SQL
SELECT active, 
       --Identify the number of active and inactive customers by COUNT()ing them.
       COUNT(active) AS num_active, 
       --Find out the differences in their average payments by using AVG().
       AVG(amount) AS avg_amount, 
       --Find out the differences in their total payments by using SUM()
       SUM(amount) AS total_amount
FROM payment AS p
INNER JOIN customer AS c
  ON p.customer_id = c.customer_id
--GROUP BY whether customer payments are active.
GROUP BY active;

-------------------------------------------------------------------------------------------
active	num_active	avg_amount	total_amount
false	3278	4.23893227577773	13895.2199999994
true	12771	4.19084566596158	53521.2899999954
-------------------------------------------------------------------------------------------
```
### Aggregating strings
You are planning to update your storefront window to demonstrate how family-friendly and multi-lingual your DVD collection is. To prepare for this you need to prepare a comma-separated list G-rated film titles by language released in 2010.

We have language and film databases
* language
    * language_id	
    * name	
    * last_update

* film
    * film_id	
    * title	
    * description	
    * release_year	
    * language_id	
    * rental_duration	
    * rental_rate	
    * length	
    * replacement_cost	
    * rating	
    * special_features
    


```SQL
SELECT name, 
       -- Return a column with a list of comma-separated film titles by using the STRING_AGG() function.
	   STRING_AGG(title, ',') AS film_titles
FROM film AS f
INNER JOIN language AS l
  ON f.language_id = l.language_id
-- limiting results with release year 2010 and rating with 'G'
WHERE release_year = 2010
  AND rating = 'G'
-- grouped by the language name
GROUP BY name;

--------------------------------------------------------------------------------------------
name	  film_titles
English	  ACE GOLDFINGER,VALLEY PACKER
Japanese	  AMISTAD MIDSUMMER,BUGSY SONG,DOCTOR GRAIL,MARRIED GO
German	  BEAUTY GREASE
Mandarin	  ATLANTIS CAUSE,AUTUMN CROW,CASUALTIES ENCINO,GARDEN ISLAND,RINGS HEARTBREAKERS,SAMURAI LION,SUICIDES SILENCE
French	  CAT CONEHEADS,DANCING FEVER,LUST LOCK
Italian	  DESPERATE TRAINSPOTTING,DWARFS ALTER,GRAPES FURY,JAWS HARRY,PACIFIC AMISTAD,PANIC CLUB
--------------------------------------------------------------------------------------------
```
### Limiting your search

You may find yourself working with tables that contain so many records that simple queries can take forever to load. This is especially challenging when you're searching for the right table to use and just need a quick result. For these scenarios the trick is to LIMIT the number of rows that your query returns. In this exercise you will practice using this function.

```SQL
SELECT * 
FROM payment
--Before we limit we ORDER BY in the DESCending amount to select highest amount payed
ORDER BY amount DESC 
LIMIT 10;

--------------------------------------------------------------------------------------------
payment_id	customer_id	rental_id	amount	payment_date
24866	237	11479	   11.99	2017-03-03 01:46:39+00:00
28814	592	3973	    11.99	2017-04-07 01:26:57+00:00
23757	116	14763	   11.99	2017-03-22 02:02:26+00:00
--------------------------------------------------------------------------------------------

```
### What tables are in your database?
You don't have to rely solely on knowing what tables exist. Instead, you can query the pg_catalog.pg_tables to list all of the tables that exist in your database.

Of course, this will list every table, including system tables so ideally, you want to limit your results to the schema where your data resides which in this case is 'public'.

Note: This system table is specific to PostgreSQL but similar tables exist for other databases (see slides).

```SQL
SELECT * 
FROM pg_catalog.pg_tables
WHERE schemaname = 'public';
--------------------------------------------------------------------------------------------
schemaname	tablename	tableowner	tablespace	hasindexes	hasrules	hastriggers	rowsecurity
public	    actor	      postgres	  null	    false	    false	    false	    false
public	    address	    postgres	  null	    false	    false	    false	    false
--------------------------------------------------------------------------------------------
```
### Determine the monthly income
Now that you know how to find the table that you need to answer a question and how to use SQL to answer that question let's practice these skills end-to-end.

How much does this business makes per month?

First, you will need to use pg_catalog.pg_tables to find the possible tables and determine which tables & columns you need to answer that question. Second, you will leverage the tools you learned in the previous chapter to prepare the answer.
```SQL
-- List all tables in the public schema
SELECT * 
FROM pg_catalog.pg_tables
WHERE schemaname = 'public';

-- Explore the tables and fill in the correct one
SELECT * 
FROM payment 
LIMIT 10;

-- Prepare the result
SELECT EXTRACT(MONTH FROM payment_date) AS month, 
       SUM(amount) AS total_payment
FROM payment
GROUP BY MONTH;

--------------------------------------------------------------------------------------------
month	total_payment
1	4781.54999999986
4	27664.4900000036
3	23886.5600000021
5	1409.15
2	9674.7599999996
Showing 5 out of 5 rows
--------------------------------------------------------------------------------------------
```
### What columns are in your database?
Just like pg_catalog.pg_tables can be incredibly helpful for listing all the tables in your database, information_schema.columns can be used to list the columns of these tables. In this exercise, you will combine these system tables to get a list of all of the columns for all your tables (in the 'public' schema).

Note: These system tables are specific to PostgreSQL but similar tables exist for other databases (see slides).
```SQL
SELECT table_name, column_name
FROM information_schema.columns
WHERE table_schema= 'public';
--------------------------------------------------------------------------------------------
table_name	   column_name
film	      film_id
rental	      rental_date
inventory	  inventory_id
customer	  last_name
payment        payment_date
film	       release_year
film	      replacement_cost
customer	  customer_id
film	      rental_rate
film	      description
address        phone
customer	   email
address	      city
customer	   active
film_actor	   actor_id
film	       length
address	       address_id
actor	       first_name
film	        rental_duration
language	     name
address	          district
rental	         return_date
--------------------------------------------------------------------------------------------
```
### A VIEW of all your columns
In this exercise you will create a new tool for finding the tables and columns you need. Using the system tables information_schema.columns and pg_catalog.pg_tables you will concatenate the list of each table's columns into a single entry.

Once you've done this you will make this query easily reusable by creating a new VIEW for it called table_columns.
```SQL
-- Create a new view called table_columns
CREATE VIEW table_columns AS
SELECT table_name, 
	   STRING_AGG(column_name, ', ') AS columns
FROM information_schema.columns
WHERE table_schema = 'public'
GROUP BY table_name
ORDER BY columns;

-- Query the newly created view table_columns
SELECT *
FROM table_columns;

--------------------------------------------------------------------------------------------
table_name	        columns
film_actor	        actor_id, film_id
category	        category, film_id
table_columns       columns, table_name
film	            film_id, release_year, replacement_cost, rental_rate, description, length, rental_duration,                       language_id, rating, title, special_features
actor	            first_name, actor_id, last_name
inventory	        inventory_id, film_id
customer	        last_name, customer_id, email, active, address_id, first_name
language	        name, language_id, last_update
payment	            payment_date, customer_id, amount, payment_id, rental_id
address	            phone, city, address_id, district, address, postal_code
rental	            rental_date, return_date, inventory_id, rental_id, customer_id
--------------------------------------------------------------------------------------------
```
### The average length of films by category
From the previous exercise you've learned that the tables film and category have the necessary information to calculate the average movie length for every category. You've also learned that they share a common field film_id which can be used to join these tables. Now you will use this information to query a list of average length for each category.
```SQL
-- Calculate the average_length for each category
SELECT category, 
	   AVG(length) AS average_length
FROM film AS f
-- Join the tables film & category
INNER JOIN category AS c
  ON f.film_id = c.film_id
GROUP BY category
-- Sort the results in ascending order by length
ORDER BY average_length DESC;
--------------------------------------------------------------------------------------------
category	average_length
Sports	128.2027027027027027
Games	127.8360655737704918
Foreign	121.6986301369863014
Drama	120.8387096774193548
Comedy	115.8275862068965517
Family	114.7826086956521739
Music	113.6470588235294118
Travel	113.3157894736842105
Horror	112.4821428571428571
Classics	111.6666666666666667
Action	111.6093750000000000
New	111.1269841269841270
Animation	111.0151515151515152
Children	109.8000000000000000
Documentary	108.7500000000000000
Sci-Fi	108.1967213114754098
--------------------------------------------------------------------------------------------
```
### Which films are most frequently rented?
Now that you've figured out the relationships between the tables and their columns, you are ready to answer the question we started with:

**Which films are most frequently rented?**

*Use the relationship diagram to answer this question.*


|film|inevntory|rental|	
|---|---|---|
|film_id|film_id|rental_id|
|release_year|inventory_id|inventory_id|
|title|--|rental_date|
|replacement_cost|--|customer_id|
......


```SQL
SELECT title, COUNT(title)
FROM film AS f
INNER JOIN inventory AS i
  ON f.film_id = i.film_id
INNER JOIN rental AS r
  ON i.inventory_id = r.inventory_id
GROUP BY title
ORDER BY count DESC;
--------------------------------------------------------------------------------------------
title	            count
BUCKET BROTHERHOOD	34
ROCKETEER MOTHER	  33
GRIT CLOCKWORK	     32
--------------------------------------------------------------------------------------------
```
Storing new data
You're planing to run a promotion on movies that won a best film academy award in the last 5 years. To do this you need to add a table in your database containing the movies which won an Oscar for best film.

The data you need for this exercise is provided in the table below:

|title	|award|
|---|---|
|'TRANSLATION SUMMER'	|'Best Film'|
|'DORADO NOTTING'|	'Best Film'|
|'MARS ROMAN'	|'Best Film'|
|'CUPBOARD SINNERS'|	'Best Film'|
|'LONELY ELEPHANT'	|'Best Film'|


```SQL
-- Create a new table called oscars
CREATE TABLE oscars (
    title VARCHAR,
    award VARCHAR
);

-- Insert the data into the oscars table
INSERT INTO oscars (title, award)
VALUES
('TRANSLATION SUMMER', 'Best Film'),
('DORADO NOTTING', 'Best Film'),
('MARS ROMAN', 'Best Film'),
('CUPBOARD SINNERS', 'Best Film'),
('LONELY ELEPHANT', 'Best Film');

-- Confirm the table was created and is populated
SELECT * 
FROM oscars;
--------------------------------------------------------------------------------------------
title	               award
TRANSLATION SUMMER   Best Film
DORADO NOTTING  	 Best Film
MARS ROMAN      	 Best Film
CUPBOARD SINNERS 	Best Film
LONELY ELEPHANT  	Best Film
--------------------------------------------------------------------------------------------
```
**Using existing data**
You are interested in identifying and storing information about films that are family-friendly. To do this, you will create a new table family_films using the data from the film table. This new table will contain a subset of films that have either the rating G or PG.
```SQL
-- Create a new table named family_films using this query
CREATE TABLE family_films AS
SELECT *
FROM film
WHERE rating IN ('G', 'PG');

```

### Update the price of rentals
You just learned that there have been some updates for the rental pricing of your films. In this exercise you will leverage the UPDATE command to modify the rental prices by increasing the rental_rate with the following logic.

All films now cost 50 cents more to rent.
R Rated films will go up by an additional 1 dollar.
```SQL
-- Increase rental_rate by 0.5 in the film table
UPDATE film
SET rental_rate = rental_rate + 0.5;

UPDATE  film
SET rental_rate = rental_rate +1
WHERE rating = 'R';

```
### Updated based on other tables
The rental company is running a promotion and needs you to lower the rental costs by 1 dollar of films who star the actors/actresses with the following last names: WILLIS, CHASE, WINSLET, GUINESS, HUDSON.

To UPDATE this data in the film table you will need to identify the film_id for these actors.
```SQL
UPDATE  film
SET rental_rate = rental_rate -1
WHERE film_id IN
  (SELECT film_id from actor AS a
   INNER JOIN film_actor AS f
      ON a.actor_id = f.actor_id
   WHERE last_name IN ('WILLIS', 'CHASE', 'WINSLET', 'GUINESS', 'HUDSON'));

```
### Delete selected records
You've discovered that some films are just not worth keeping your inventory, for cases where the replacement_cost is greater than 25 dollars. As such you'd like to remove them from you film table.
```SQL
-- Delete films that cost most than 25 dollars
DELETE FROM film
WHERE replacement_cost > 25

```
# A family friendly video store
Your company has decided to become a family friendly store. As such, all R & NC-17 movies will be cleared from the inventory. You will take the steps necessary to clear these films from both the inventory and the film tables.
```SQL
-- Use the list of film_id values to DELETE all R & NC-17 rated films from inventory.
DELETE FROM inventory
WHERE film_id IN (
  SELECT film_id FROM film
  WHERE rating IN ('R', 'NC-17')
);

-- Delete records from the `film` table that are either rated as R or NC-17.
DELETE FROM film
WHERE rating IN ('R', 'NC-17');

```

```SQL

--------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------
```
```SQL

--------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------
```